In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [25]:
hoodie_prompt = """You are a fashion styling expert.
Compare the hoodie in the reference snap photo with the hoodie in the product image,
and generate a textual requirement that a user might input when searching for a target product.

Input Data
Image 1: Reference hoodie snap photo
Image 2: Target hoodie product image

Generation Rules
When a user searches for a hoodie similar to the reference image, they select their preferences based on four criteria.
For each criterion, only one option can be chosen.
The output should be a structured phrase, not conversational text.
Preferably, do not select more than three requirements.
**If no change is required for a specific criterion, select "no requirements."
If "no requirements" is selected, exclude that criterion from the generated text.
ex) If both the hoodies in the snapshot and the product are plain, you shouldn't say anything about the design.**

Criteria

1. Design
No requirements → Exclude
Plain → "has no graphic and pattern"
Added graphic → "has added graphic"
Striped pattern → "has a striped pattern"
Checkered pattern → "has a checkered pattern"
Camouflage pattern → "has a camouflage pattern"

2. Color
No requirements → Exclude
Black → "is black"
White → "is white"
Gray → "is gray"
Blue → "is blue"
Red → "is red"
Other colors → "is {selected color}"

3. Length
No requirements → Exclude
Cropped → "is cropped length"
Regular → "is regular length"
Long → "is long length"

4. Fit
No requirements → Exclude
Slim fit → "is slim fit"
Regular fit → "is regular fit"
Oversized fit → "is oversized fit"

Example Outputs
If the user selects Design: No requirements, Color: Black, Length: No requirements, Fit: Oversized
"is black and oversized fit"

If the user selects Design: Removed print, Color: Navy, Length: Long, Fit: slim fit
"has no print, is navy, and long length with slim fit"

If the user selects Design: No requirements, Color: No requirements, Length: Cropped, Fit: No requirements
"is cropped length"
"""

coat_prompt = """You are a fashion styling expert.
Compare the coat in the reference snap photo with the coat in the product image,
and generate a textual requirement that a user might input when searching for a target product.

Input Data
Image 1: Reference coat snap photo
Image 2: Target coat product image

Generation Rules
When a user searches for a hoodie similar to the reference image, they select their preferences based on four criteria.
For each criterion, only one option can be chosen.
The output should be a structured phrase, not conversational text.
Preferably, do not select more than three requirements.
**If no change is required for a specific criterion, select "no requirements."
If "no requirements" is selected, exclude that criterion from the generated text.
ex) If both the coats in the snapshot and the product are plain, you shouldn't say anything about the design.**

Criteria

1. Design
No requirements → Exclude
Plain → "is plain"
herringbone pattern → "has a herringbone pattern"
Striped pattern → "has a striped pattern"
Checkered pattern → "has a checkered pattern"

2. Color
No requirements → Exclude
Black → "is black"
White → "is white"
Gray → "is gray"
Other colors → "is {selected color}"

3. Length
No requirements → Exclude
half → "is half-length"
Regular → "is regular length"
Long → "is long length"

4. Fit
No requirements → Exclude
Slim fit → "is slim fit"
Regular fit → "is regular fit"
Oversized fit → "is oversized fit"

Example Outputs
If the user selects Design: No requirements, Color: Black, Length: No requirements, Fit: Oversized
"is black and oversized fit"

If the user selects Design: plain, Color: Navy, Length: Long, Fit: No requirements
"is plain, navy, and long length"

If the user selects Design: No requirements, Color: No requirements, Length: half, Fit: No requirements
"is half-length"
"""

jeans_prompt = """You are a fashion styling expert.
Compare the jeans in the reference snap photo with the jeans in the product image,
and generate a textual requirement that a user might input when searching for a target product.

Input Data
Image 1: Reference jeans snap photo
Image 2: Target jeans product image

Generation Rules
When a user searches for a hoodie similar to the reference image, they select their preferences based on four criteria.
For each criterion, only one option can be chosen.
The output should be a structured phrase, not conversational text.
Preferably, do not select more than three requirements.
**If no change is required for a specific criterion, select "no requirements."
If "no requirements" is selected, exclude that criterion from the generated text.
ex) If both the jeans in the snapshot and the product are regular, you shouldn't say anything about the design.**

Criteria

1. Design
No requirements → Exclude
Regular → "has a regular design"
Washed → "has a washed effect"
Ripped → "has a ripped design"

2. Color
No requirements → Exclude
Black → "is black"
blue → "is blue"
Light blue → "is light blue"
Other colors → "is {selected color}"

3. Length
No requirements → Exclude
longer → "is slightly longer length"
shorter → "is slightly shorter length"

4. Fit
No requirements → Exclude
Skinny fit → "is skinny fit"
Slim fit → "is slim fit"
Straight fit → "is straight fit"
Tapered fit → "is tapered fit"
Wide fit → "is wide fit"
Bootcut fit → "is bootcut fit"
Loose fit → "is loose fit"

Example Outputs
If the user selects Design: No requirements, Color: Black, Length: No requirements, Fit: wide fit
"is black and wide fit"

If the user selects Design: Washed, Color: dark blue, Length: Longer, Fit: No requirements
"has a washed effect, and is dark blue, and slightly longer length"

If the user selects Design: No requirements, Color: No requirements, Length: No requirements, Fit: tapered fit
"is tapered fit"
"""

In [26]:
import base64
from PIL import Image, ImageDraw, ImageFont
import io

def encode_image_with_text(image_path, text, position=(10, 10), font_size=30, text_color="white", bg_color="black"):
    # 이미지 열기
    image = Image.open(image_path)

    # 이미지에 텍스트 추가
    draw = ImageDraw.Draw(image)

    # 폰트 설정 (시스템에 따라 폰트 경로를 변경해야 할 수도 있음)
    font = ImageFont.truetype("DejaVuSans.ttf", font_size)

    # 최신 Pillow 방식으로 텍스트 크기 계산
    text_bbox = font.getbbox(text)  # (left, top, right, bottom) 반환
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    
    padding = 5  # 여백 추가

    # 배경 사각형 그리기 (텍스트 주변에 padding 추가)
    background_position = [
        (position[0] - padding, position[1] - padding),  # 좌상단
        (position[0] + text_width + padding, position[1] + text_height + padding)  # 우하단
    ]
    draw.rectangle(background_position, fill=bg_color)

    # 텍스트 그리기
    draw.text(position, text, fill=text_color, font=font)

    # 메모리에 이미지 저장 후 Base64 인코딩
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')


In [27]:
import os
import json
import base64
import random
from glob import glob
from tqdm import tqdm
import openai

# 1) OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = ""  # Enter your OpenAI Key

def generate_caption_from_images(ref_img_path, tgt_img_path, category):
    print(ref_img_path, tgt_img_path)
    base64_reference_image = encode_image_with_text(ref_img_path, "snap photo")
    base64_target_image = encode_image_with_text(tgt_img_path, "target product")

    if category == 'hoodie':
        prompt = hoodie_prompt
    elif category == 'coat':
        prompt = coat_prompt
    elif category == 'jeans':
        prompt = jeans_prompt

    # prompt += '\n**Give me only one english query similar to the examples above, and keep it concise.**'

    response = openai.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": prompt,
                },
                {
                "type": "image_url",
                "image_url": {
                    "url":  f"data:image/jpeg;base64,{base64_reference_image}"
                },
                },
                {
                "type": "image_url",
                "image_url": {
                    "url":  f"data:image/jpeg;base64,{base64_target_image}"
                },
                },
            ],
            }
        ],
    )

    print(response.choices[0].message.content)
    return response.choices[0].message.content


# ---------------------------------
# 메인 로직
# ---------------------------------
def build_composed_image_retrieval_dataset(
    snap_root,
    goods_root,
    output_root,
    categories=["hoodie", "coat", "jeans"],
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    max_pairs_per_category=500
):
    """
    FashionIQ 형식에 맞춘 Composed Image Retrieval Dataset 생성.

    :param snap_root: snap_images 폴더 경로 (reference)
    :param goods_root: goods_images 폴더 경로 (target)
    :param output_root: 최종 데이터셋 출력 경로 (fashionIQ_dataset 처럼)
    :param categories: 카테고리 리스트
    :param train_ratio, val_ratio, test_ratio: 데이터셋 분할 비율
    """

    # 예: fashionIQ_dataset/captions, fashionIQ_dataset/image_splits, fashionIQ_dataset/images
    captions_dir = os.path.join(output_root, "captions")
    splits_dir   = os.path.join(output_root, "image_splits")
    images_dir   = os.path.join(output_root, "images")

    os.makedirs(captions_dir, exist_ok=True)
    os.makedirs(splits_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)

    for category in categories:
        # 1) 폴더 구조에서 이미지 목록 읽어오기
        snap_cat_dir  = os.path.join(snap_root, category)   # snap_images/<category>
        goods_cat_dir = os.path.join(goods_root, category)  # goods_images/<category>

        # snap과 goods가 각각 존재하는 파일명 쌍을 찾아서 pairs 리스트로 만든다
        snap_files = glob(os.path.join(snap_cat_dir, "*.jpg"))
        goods_files = glob(os.path.join(goods_cat_dir, "*.jpg"))

        all_pairs = []
        for s_fp in snap_files:
            for g_fp in goods_files:
                all_pairs.append((s_fp, g_fp))

        if len(all_pairs) == 0:
            print(f"[WARNING] No possible pairs for category '{category}'")
            continue

        # 이제 all_pairs 중에서 최대 max_pairs_per_category개만 랜덤 추출
        if len(all_pairs) > max_pairs_per_category:
            pairs = random.sample(all_pairs, max_pairs_per_category)
        else:
            pairs = all_pairs

        # 2) train/val/test split
        random.shuffle(pairs)
        total = len(pairs)
        train_count = int(total * train_ratio)
        val_count   = int(total * val_ratio)
        test_count  = total - (train_count + val_count)

        train_pairs = pairs[:train_count]
        val_pairs   = pairs[train_count:train_count+val_count]
        test_pairs  = pairs[train_count+val_count:]

        # 3) captions JSON (train, val, test)
        train_json = []
        val_json   = []
        test_json  = []

        # 4) image_splits JSON (train, val, test) -> Image ID(파일명) 리스트
        train_splits = []
        val_splits   = []
        test_splits  = []

        for split_name, split_pairs in zip(
            ["train", "val", "test"],
            [train_pairs, val_pairs, test_pairs]
        ):
            # 해당 split에 대해 JSON 구성
            split_data_list = []
            split_image_ids = []

            for ref_fp, tgt_fp in tqdm(split_pairs, desc=f"{category}-{split_name}"):
                ref_name = os.path.basename(ref_fp)  # reference filename
                tgt_name = os.path.basename(tgt_fp)  # target filename

                # 캡션 생성 (실제로는 OpenAI API 호출해야 함)
                captions = generate_caption_from_images(ref_fp, tgt_fp, category)

                # FashionIQ 포맷: { "target": "B0084Y8XIU", "candidate": "B005X4PL1G", "captions": [...], }
                # 여기서는 파일명을 그대로 사용하되, 확장자(.jpg) 없이 ID만 사용한다 가정
                # 혹은 어떤 해시나 임의 ID로 사용해도 무방.
                target_id    = os.path.splitext(tgt_name)[0]
                candidate_id = os.path.splitext(ref_name)[0]

                split_data_list.append({
                    "target": target_id,
                    "candidate": candidate_id,
                    "captions": captions
                })

                # 이미지 스플릿에는 타겟 이미지 ID를 넣는다고 가정 (fashionIQ 예시)
                split_image_ids.append(target_id)

                # 5) 이미지 파일 복사/링크 (원본 이미지 -> fashionIQ_dataset/images 폴더)
                # 실제로는 copy나 symlink를 생성해주면 좋음
                # 이 예시에선 단순히 "이 폴더에 하드카피" 하거나, 이미지를 옮기는 건 생략
                # 필요하면 아래 코드 활성화:
                # import shutil
                # shutil.copy2(ref_fp, os.path.join(images_dir, ref_name))
                # shutil.copy2(tgt_fp, os.path.join(images_dir, tgt_name))
                # 혹은 os.symlink(...) 로 심볼릭 링크

            # split_data_list를 split_name에 따라 분기
            if split_name == "train":
                train_json = split_data_list
                train_splits = split_image_ids
            elif split_name == "val":
                val_json = split_data_list
                val_splits = split_image_ids
            elif split_name == "test":
                test_json = split_data_list
                test_splits = split_image_ids

        # 6) 카테고리별로 captions json 저장
        cap_train_path = os.path.join(captions_dir, f"cap.{category}.train.json")
        cap_val_path   = os.path.join(captions_dir, f"cap.{category}.val.json")
        cap_test_path  = os.path.join(captions_dir, f"cap.{category}.test.json")

        with open(cap_train_path, "w", encoding="utf-8") as f:
            json.dump(train_json, f, indent=2, ensure_ascii=False)
        with open(cap_val_path, "w", encoding="utf-8") as f:
            json.dump(val_json, f, indent=2, ensure_ascii=False)
        with open(cap_test_path, "w", encoding="utf-8") as f:
            json.dump(test_json, f, indent=2, ensure_ascii=False)

        # 7) 카테고리별로 split json 저장
        split_train_path = os.path.join(splits_dir, f"split.{category}.train.json")
        split_val_path   = os.path.join(splits_dir, f"split.{category}.val.json")
        split_test_path  = os.path.join(splits_dir, f"split.{category}.test.json")

        with open(split_train_path, "w", encoding="utf-8") as f:
            json.dump(train_splits, f, indent=2, ensure_ascii=False)
        with open(split_val_path, "w", encoding="utf-8") as f:
            json.dump(val_splits, f, indent=2, ensure_ascii=False)
        with open(split_test_path, "w", encoding="utf-8") as f:
            json.dump(test_splits, f, indent=2, ensure_ascii=False)

        print(f"[INFO] Finished processing category: {category}")

if __name__ == "__main__":
    # 예시 경로 설정 (적절히 변경)
    snap_images_root  = "/home/sejong/Desktop/junseok/SPRC/musinsa/snap/men"
    goods_images_root = "/home/sejong/Desktop/junseok/SPRC/musinsa/goods/men"
    output_root       = "/home/sejong/Desktop/junseok/SPRC/musinsa/men"

    # 실제 코드 실행
    build_composed_image_retrieval_dataset(
        snap_root=snap_images_root,
        goods_root=goods_images_root,
        output_root=output_root,
        categories=["hoodie", "coat", "jeans"],
        train_ratio=0.8,
        val_ratio=0.1,
        test_ratio=0.1,
        max_pairs_per_category=500
    )

hoodie-train:   0%|          | 0/400 [00:00<?, ?it/s]

/home/sejong/Desktop/junseok/SPRC/musinsa/snap/men/hoodie/list.staff_6625cc81d9feb.jpg /home/sejong/Desktop/junseok/SPRC/musinsa/goods/men/hoodie/더티워싱 스터드 후드 집업 [차콜].jpg


hoodie-train:   0%|          | 1/400 [00:04<33:05,  4.98s/it]

"is black and oversized fit"
/home/sejong/Desktop/junseok/SPRC/musinsa/snap/men/hoodie/438aeb24a6b646289a8db177ee8311a5.jpg /home/sejong/Desktop/junseok/SPRC/musinsa/goods/men/hoodie/컴피 리벳 하이 넥 스웻 집업 [BLACK].jpg


hoodie-train:   0%|          | 2/400 [00:09<31:42,  4.78s/it]

...

"has a ripped design, and is dark blue, and is wide fit"
/home/sejong/Desktop/junseok/SPRC/musinsa/snap/men/jeans/ba605c2845344ce18a88099ba96ec0f2.jpg /home/sejong/Desktop/junseok/SPRC/musinsa/goods/men/jeans/[무료반품] 린넨 데님 워시드 밴딩 팬츠 - 3 COLOR.jpg


jeans-test:  98%|█████████▊| 49/50 [03:11<00:03,  3.79s/it]

"has a washed effect, and is blue, and is loose fit"
/home/sejong/Desktop/junseok/SPRC/musinsa/snap/men/jeans/f8fddad247d840e2aea557604df77632.jpg /home/sejong/Desktop/junseok/SPRC/musinsa/goods/men/jeans/USN Side Seamless Denim Pants Mid Indigo.jpg


jeans-test: 100%|██████████| 50/50 [03:15<00:00,  3.90s/it]

"is blue and loose fit"
[INFO] Finished processing category: jeans
